# 1. Import libraries

In [1]:
# data manipulation
import numpy as np
import pandas as pd
import re
from datetime import datetime

# data collection
import requests
from bs4 import BeautifulSoup

# progress display
from tqdm import tqdm

# 2. Data collection and preparation

### Articles

In [2]:
# Create Dataset
dataset = pd.DataFrame(columns=['Date', 'Title', 'Content'])

### Link searching ###
# data from the last 5 years due to a rather large dataset
history = [date.strftime("%Y-%m-%d") for date in pd.date_range(start="2017-01-01", end=datetime.today())]
for date in tqdm(history):
    # print(date)
    url = 'https://forsal.pl/archiwum/'+str(date)
#     print(url)
    f = requests.get(url)
    soup = BeautifulSoup(f.text, 'html.parser')
    mat = str(soup)
    links = (re.findall(r"<a.*? href=\"(.*?)\".*?>(.*?)</a>", mat)[:])
    for link in links:
        if (('artykuly' in link[0])&('https://www.gazetaprawna.pl/artykuly' not in link[0])&('swiat' not in link[0]))&\
            (('notowania' in link[0])|('finanse/gielda' in link[0])):
            # print(link[0])
            url2 = link[0]
            f = requests.get(url2)
            soup = BeautifulSoup(f.text, 'html.parser')
            title = link[1]
            # print(title)
            # print(date)
            try:
                content = soup.find_all(class_='leadDiv')[0].text[:]+' '+soup.find_all(class_='articleBody')[0].text.split('>>>')[0]
            except:
                try:
                    content = soup.find_all(class_='lead')[0].text[:]+' '+soup.find_all(class_='articleBody')[0].text.split('>>>')[0]
                except:
                    continue
            # print(content)
            # print()
            dataset_single_article = pd.DataFrame(({'Date':[date], 'Title':title, 'Content':content}))
            dataset = pd.concat([dataset, dataset_single_article], axis=0)

dataset.reset_index(drop=True, inplace=True)
dataset.to_csv('Articles.csv')
dataset

100%|████████████████████████████████████████████████████████████████████████████| 1842/1842 [1:28:54<00:00,  2.90s/it]


,Date,Title,Content
0,2017-01-02,"DZIEŃ NA GPW: WIG 20 lekko zyskał, w górę posz...",\n Pierwszą sesję w tym roku WI...
1,2017-01-04,Niezatapialna kanclerz. Poprawiają się notowan...,\n Ponad dwa tygodnie po zamachu terror...
2,2017-01-12,Co zrobi Fitch i Moody's? Oto scenariusze dla ...,\n W piątek 13. stycznia dwie wiodące a...
3,2017-01-16,DZIEŃ NA GPW: Po początkowych spadkach indeksy...,\n Indeksy GPW zakończyły ponie...
4,2017-02-02,Nominacja dla Schulza strzałem w dziesiątkę. N...,\n Po nominacji Martina Schulza na kand...
...,...,...,...
1467,2022-01-14,”Moscow Times”: Spadki na giełdzie w Moskwie p...,\n Kurs rubla oraz ceny akcji c...
1468,2022-01-14,Władysław Szczepkowski powołany na p.o. prezes...,\n Rada nadzorcza PKP Cargo del...
1469,2022-01-14,Ropa w USA drożeje już 4 tydzień z rzędu,\n Ropa naftowa w USA nieznaczn...
1470,2022-01-14,Główne indeksy na Wall Street zniżkują,\n Czwartkowa sesja na Wall Str...


In [3]:
dataset['Date'].unique()

array(['2017-01-02', '2017-01-04', '2017-01-12', '2017-01-16',
       '2017-02-02', '2017-02-09', '2017-02-24', '2017-02-28',
       '2017-03-01', '2017-03-07', '2017-03-17', '2017-03-25',
       '2017-03-27', '2017-03-28', '2017-04-04', '2017-04-11',
       '2017-05-02', '2017-05-05', '2017-05-07', '2017-05-09',
       '2017-05-30', '2017-06-08', '2017-07-03', '2017-07-11',
       '2017-07-18', '2017-07-24', '2017-07-27', '2017-08-07',
       '2017-08-08', '2017-08-09', '2017-08-10', '2017-08-21',
       '2017-08-22', '2017-08-23', '2017-08-25', '2017-08-28',
       '2017-09-01', '2017-09-04', '2017-09-19', '2017-09-20',
       '2017-09-22', '2017-09-27', '2017-10-11', '2017-10-17',
       '2017-10-23', '2017-10-26', '2017-10-31', '2017-11-02',
       '2017-11-03', '2017-11-06', '2017-11-07', '2017-11-10',
       '2017-11-13', '2017-11-14', '2017-11-24', '2017-11-29',
       '2017-12-04', '2017-12-22', '2017-12-28', '2018-01-09',
       '2018-01-12', '2018-01-25', '2018-02-05', '2018-

### Stock market - WIG 20 historic price

In [4]:
wig_20 = pd.read_csv('https://stooq.pl/q/d/l/?s=wig20&i=d')
wig_20.to_csv('WIG_20.csv')
wig_20

,Data,Otwarcie,Najwyzszy,Najnizszy,Zamkniecie,Wolumen
0,1991-04-16,100.00,100.00,100.00,100.00,325
1,1991-04-23,95.70,95.70,95.70,95.70,5905
2,1991-04-30,93.50,93.50,93.50,93.50,7162
3,1991-05-14,92.90,92.90,92.90,92.90,18300
4,1991-05-21,95.50,95.50,95.50,95.50,14750
...,...,...,...,...,...,...
7235,2022-01-10,2311.34,2328.41,2289.59,2291.23,22612381
7236,2022-01-11,2300.38,2345.44,2296.03,2344.45,30819446
7237,2022-01-12,2350.99,2411.11,2350.99,2411.11,46032724
7238,2022-01-13,2406.83,2429.03,2397.64,2407.62,36970690
